%%HTML
<link rel="stylesheet" type="text/css" href="custom.css">

from IPython.display import HTML, display

def set_background(color):    
    script = (
        "var cell = this.closest('.jp-CodeCell');"
        "var editor = cell.querySelector('.jp-Editor');"
        "editor.style.background='{}';"
        "this.parentNode.removeChild(this)"
    ).format(color)
    
    
display(HTML('<img src onerror="{}" style="display:none">'.format(script)))


# Prevendo o imprevisível: agregação de pesquisas eleitorais em 2022

Se você estiver lendo a versão interativa desta apresentação no binder, pressione `espaço` para mover para o próximo slide.

# Para aqueles executando localmente
Remova todos headers %%capture para obter as imagens dos resultados

In [1]:
import matplotlib.pyplot as plt
from scipy import stats, optimize
import numpy as np
import pandas as pd
import jotaviz
from jotaviz import styles
from datetime import date, datetime, timedelta
np.set_printoptions(precision=2, floatmode="fixed")

In [2]:
def hide_code_in_slideshow():   
    from IPython import display
    import binascii
    import os
    uid = binascii.hexlify(os.urandom(8)).decode()    
    html = """<div id="%s"></div>
    <script type="text/javascript">
        $(function(){
            var p = $("#%s");
            if (p.length==0) return;
            while (!p.hasClass("cell")) {
                p=p.parent();
                if (p.prop("tagName") =="body") return;
            }
            var cell = p;
            cell.find(".input").addClass("hide-in-slideshow")
        });
    </script>""" % (uid, uid)
    display.display_html(html, raw=True)

## Apresentação

Esta apresentação será apresentada de uma forma que não exigirá conhecimento profundo de nenhum dos tópicos técnicos, quero apenas mostrar a intuição, problemas, agregação e modelos de projeção.

### Introdução
   * Por que as pesquisas variam tanto?


# Quem sou eu?
### Analista de dados no JOTA em Brasília
### Twitter e github: dmarcelinobr
### E-mail: daniel.marcelino@jota.info


<right><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/template-colunistas-pro-35.png?raw=true" alt="Crédito: Raoni Arruda" style="height: 300px;"/> </right>

# As pesquisas eleitorais...
## Para as consultorias e institutos de opinião as pesquisas funcionam assim:


<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/target-with-dart-flat.jpg?raw=true" alt="" style="height: 300px;"/> </center>

# As pesquisas eleitorais...
## Mas na prática... bem, elas funcionam mais ou menos assim:


<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/archery-target-ring.png?raw=true" alt="" style="height: 400px;"/> </center>

# Teoria vs. prática

### René Descartes: toda explicação científica se deve expressar em termos de quantidades precisas e matematicamente definidas (dúvida cartesiana).

### Carl Hempel: as ciências são modelos da realidade, explanatórios e preditivos, em aproximações sucessivas da realidade estudada. 

### Thomas Kuhn: a ciência se desenvolve através de paradigmas que se sucedem na explicação dos fatos.

### É por isso que a Mecânica Newtoniana é suficiente para o lançamento de satélites e previsão da órbita dos planetas (exceto Mercúrio).

# Mutatis mutandis

### As pesquisas refletem uma dada opinião num certo momento. 

### O ''Pai-Nosso'' das consultorias diz que as pesquisas: **são uma fotografia do momento**. 

### Por se tratar de opinião, **<font color='red'>matéria-prima volátil</font>**, o grau de variação entre um momento e outro também está envolto em um quadro móvel de circunstâncias.

### Qualquer nova informação **pode alterar ou não** esse quadro.


# Pesquisas são incertas por natureza

### Todo modelo científico se **respalda na explicação dos resultados** e na **<font color='blue'>previsão</font>** do que possa vir a ocorrer. Assim é na física, na química e na biologia, dentre alguns graus possíveis de previsibilidade.

### Na estatística, e consequentemente nas pesquisas eleitorais, os modelos são de **menor previsibilidade do que na física**, devido à **<font color='red'>margem de erro</font>** ou de variação aleatória inerente ao método amostral e à volatilidade das opiniões e intenção de voto. 

### Lembre-se: elas se baseiam na teoria estatística da **representatividade** de uma amostra, dentro de um universo. O que ocorre quando temos várias representações desse universo?

# Erro vs error

### É importante observar outros aspectos, inclusive linguísticos. Em português, o termo **<font color='red'>erro</font>** é carregado de conotação negativa. Já em inglês, o termo **<font color='blue'>error</font>**, utilizado na expressão **margin of error**, difere do termo **<font color='red'>mistake</font>**, que leva a conotação negativa na língua inglesa. 

### A **margem de erro**, ou **margin of error**, deve ser entendida como **margem de variação técnica** antecipadamente prevista em uma amostra, e não como **erro** ocorrido após uma medição.

# O mundo real é mais complexo
### Parece difícil medir a intenção de votos para Lula em certos momentos

<right><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/voto_espontaneo_lula.png?raw=true" alt="Daniel" style="height: 400px;"/> </right>

# O mundo real é mais complexo
### Parece difícil medir a intenção de votos para Bolsonaro também  

<right><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/voto_espontaneo_bolsonaro.png?raw=true" alt="Daniel" style="height: 400px;"/> </right>

# E pode piorar ainda mais...
### House effects, mode effects, etc

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/lula_estimulado.png?raw=true" alt="Daniel" style="height: 400px;"/> </center>

# E pode piorar ainda mais...
### House effects, mode effects, etc

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/bolsonaro_estimulado.png?raw=true" alt="Daniel" style="height: 400px;"/> </center>

# A teoria funciona, mas... 

### Eleitores não são como bolas coloridas usadas nos experimentos de probabilidade descritos em livros didáticos; as bolas azuis e vermelhas nunca mundam de cor neles.

### Mas eleitores **<font color='red'>podem ou não mudar de opinião</font>**, de acordo com estímulos que recebem. As campanhas estão aí pra isso! 

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/porcentagem.gif?raw=true" alt="mutatis mutandis" style="height: 300px;"/> </center>

# Mas afinal, é possível prever a próxima eleição?
<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/tela_agregadores.png?raw=true" alt="mutatis mutandis" style="height: 500px;"/> </center>

Na minha experiência, modelos bayesianos são particularmente poderosos em caracterizar a incerteza, mesmo em situações com uma pequena quantidade de dados. 

## Sim, mas é preciso modelar os dados

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/dados_agregadores.png?raw=true" alt="mutatis mutandis" style="height: 440px;"/> </center>

# O que a experiência me ensionou 

### - Um modelo de previsão eleitoral deve ser **<font color='orange'>probabilístico</font>**, não determinístico.

### - Um modelo deve ser **<font color='purple'>empírico</font>**.

### - Um modelo deve ser **<font color='green'>sensível</font>** às mudanças nos resultados medidos pelas pesquisas.

### - Um modelo deve ser **sensível** às mudanças no **tempo certo**.

# Os modelos usam simulações com cadeias de Markov

### Uma cadeia de Markov é uma sequência de eventos onde a probabilidade de **futuro** apenas depende do **presente**

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/mcmc_lula.png?raw=true" alt="Primeiro turno" style="height: 300px;"/> </center>

Se uma pesquisa encontrou que Lula tem 40% hoje, amanhã Lula terá essa mesma taxa com 70% de probabilidade ou uma taxa maior/menor com probabilidade de 30%.

# As pesquisas são realizações de uma distribuição Binomial ou Multinomial

### Para cada sondagem, um número **n** de eleitores são entrevistados, e uma parcela deles (**y**) revela que prefere o  candidato **<font color='red'>A</font>** ou **<font color='blue'>B</font>**.

### Então, tecnicamente temos que: **y ∼ Binomial(n, p)**, onde **p** é a parcela de eleitores que prefere um candidato entre os demais.

### Podemos usar generalizações dessa lógica para eleições com mais de dois candidatos.

# Exemplos de modelos
### Previsão eleitoral
<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/primeiro_turno_model.png?raw=true" alt="mutatis mutandis" style="height: 400px;"/> </center>

# Exemplos de modelos
### Impacto do Auxílio Brasil na aprovação
<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/auxilio_brasil_model_1T-VT.png?raw=true" alt="mutatis mutandis" style="height: 400px;"/> </center>

# Exemplos de modelos
### Popularidade do presidente
<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/popularidade_hierarquica_zerosum.png?raw=true" alt="mutatis mutandis" style="height: 400px;"/> </center>

# Exemplos de modelos
### Efeito do preço do dólar na popularidade do presidente
<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/cambio_latent_gp.png?raw=true" alt="mutatis mutandis" style="height: 400px;"/> </center>

## Transformamos essa visão em linhas de código

### O modelo é essencialmente um programa de computador como este:

In [17]:
with pm.Model() as primeiro_turno_model:

phi = pm.Beta('phi', alpha=alpha, beta=beta)

kappa_log = pm.HalfNormal('kappa_log', sigma=10)
kappa = pm.Deterministic('kappa', tt.exp(kappa_log))

theta = pm.Beta(
    'p', 
    alpha=phi*kappa, 
    beta=(1.0-phi)*kappa, 
    shape=len(candidato_polls)
)

polls = pm.Binomial(
    'y', 
    n=candidato_polls['amostra'], 
    p=theta, 
    observed=candidato_polls['votos']
)

# Eu ainda não contei o que acontece por debaixo dos panos

## Os resultados das pesquisas são (re)calibrados com base em três fatores:

### 1 - Há quanto **tempo** a pesquisa foi conduzida.

### 2 - A **quantidade** de eleitores ouvidos na pesquisa (tamanho da amostra).

### 3 - A **avaliação** que fazemos das empresas considerando a **precisão** dos resultados na eleição anterior. ⚠️

# Há quanto tempo a pesquisa foi conduzida

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/recencia.png?raw=true" alt="" style="height: 400px;"/> </center>

- A penalidade se torna mais rígida. Isto é, mais ênfase é colocada na recência quanto mais nos aproximamos da eleição.

- Por outro lado, na minha experiência em acompanhar esse tipo informação eu acredito que a mídia de notícias dá muita ênfase à recência do que deveria. É muito melhor olhar para uma gama mais ampla de pesquisas do que focar apenas no último dado publicado.

# Como isso funciona na prática

### Pesquisa Atlas Intel realizada nos dias 27 a 29 de novembro com 4.401 entrevistados:  

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/recencia_pond.png?raw=true" alt="" style="height: 400px;"/> </center>

# Tamanho da amostra utilizada

### Tamanho importa! As pesquisas que ouvem mais eleitores devem receber algum crédito por isso, não acha?

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/tamanho_importa.png?raw=true" alt="" style="height: 390px;"/> </center>

# Tamanho da amostra utilizada

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/tamanho_amostra.png?raw=true" alt="" style="height: 420px;"/> </center>

# Tamanho da amostra utilizada
### ... Mas os ganhos precisam ser marginais

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/peso_pesquisa.png?raw=true" alt="" style="height: 420px;"/> </center>

O ajuste é um pouco assimétrico: as sondagens com amostras muito grandes não recebem tanto crédito extra pelo modelo, e aquelas com amostras especialmente pequenas são 'punidas'

# Como isso funciona na prática

### Pesquisa Ipespe realizada nos dias 22 a 24 de novembro com 1.000 entrevistados:    

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/ex_Ipespe.png?raw=true" alt="" style="height: 400px;"/> </center>

# Avaliação da precisão da empresa

### É preciso dar crédito extra para quem se esforça em produzir levantamentos de qualidade, responsabilidade e transparência.

#### * Se algum instituto acertou o resultado dentro da margem de erro?

#### * A tendência global das pesquisa estava certa?

#### * A performance da média do mercado?

#### * Se o último ano foi pior ou melhor do que os anos anteriores?


# Como isso funciona na prática

### Pesquisa Ipespe realizada nos dias 22 a 24 de novembro com 1.000 entrevistados, tem nota 0,950:   

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/exp_precisao_ipesep.png?raw=true" alt="" style="height: 400px;"/> </center>

# Como isso funciona na prática

### Ex: pesquisa Brasmarket realizada entre os dias 05 a 07 de dezembro com 4.600 entrevistados, nota 0,150:  

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/exp_precisao_basmarket.png?raw=true" alt="" style="height: 400px;"/> </center>

# Resultados

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/lula_vs_outros.png?raw=true" alt="" style="height: 450px;"/> </center>

# Resultados

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/lula_bolsonaro.png?raw=true" alt="" style="height: 450px;"/> </center>

# Resultados

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/lula_bolsonaro_2t.png?raw=true" alt="" style="height: 450px;"/> </center>

# Resultados

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/agregador_2022_logo.png?raw=true" alt="" style="height: 475px;"/> </center>

# Resultados

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/margem_macrorregiao.png?raw=true" alt="" style="height: 475px;"/> </center>

# Resultados

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/jan-jun-regioes.png?raw=true" alt="" style="height: 475px;"/> </center>

# Resultados

<center><img src="https://github.com/dmarcelinobr/pollbase/blob/master/presentation/img/jan-jun-renda.png?raw=true" alt="" style="height: 475px;"/> </center>

# Muito obrigado!

In [18]:
# Set seed back to zero for rest of notebook
ELECTION_DATE = datetime.datetime(year=2022, month=10, day=2)
TODAY = date.today().isoformat()

countdown = ELECTION_DATE - datetime.datetime.now()

print(f"Countdown to Election BR 2022: {countdown}")

In [18]:
Countdown to Election BR 2022: 121 days